# Advanced BoW model (TF-IDF vectorization)

TF-IDF model using a distinctive terms vectorization.

In [1]:
import sys
import os
import random
import numpy as np

# Get the current directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Add the parent directory to the path if needed
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(parent_dir)
print(f"Added to path: {parent_dir}")

# set global seeds for reproducability
random.seed(161)
np.random.seed(161)
os.environ['PYTHONHASHSEED'] = '161'

Current working directory: c:\Users\felix\Documents\GIT\Hertie\PACT-ML\modules
Added to path: c:\Users\felix\Documents\GIT\Hertie\PACT-ML


## Data preparation

In [2]:
import pandas as pd
from pyreadr import read_r
from modules.helpers.validity_check import fuzzy_match_report_key

para_data = pd.read_csv("..\data\PACT_paragraphs_training.csv")

report_data = pd.read_csv("..\data\paragraphs.csv", sep=';')

report_data["matchingKey"] = report_data["report_namePKO"].str.replace('/', '_')

# reduced data to 7 target categories with most codings
target_categories = [
    "PoliceReform",
    "Operations_PatrolsInterventions",
    "StateAdministration",
    "RefugeeAssistance",
    "ElectionAssistance",
    "LegalReform",
    "CivilSocietyAssistance"
]

report_data = report_data[["matchingKey", "paragraphNumber"] + target_categories]

report_data[target_categories] = report_data[target_categories].map(lambda x: isinstance(x, str))


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

merged_data = pd.merge(
    para_data,
    report_data,
    on=["matchingKey", "paragraphNumber"],
    how="left"
)

merged_data.to_csv("../data/merged_data.csv", index=False)

# Check the shape of the merged data
print(f"Shape of merged data: {merged_data.shape}")

# vectorizer = CountVectorizer(stop_words='english', min_df=5, max_df=0.95)
# X = vectorizer.fit_transform(merged_data["paragraph"])

tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=5, max_df=0.95)
X = tfidf_vectorizer.fit_transform(merged_data['paragraph'])

# df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df_tfidf = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# print(f"Number of features for Count Vectorizer: {df_bow.shape[1]}")
print(f"Number of features for TF-IDF: {df_tfidf.shape[1]}")

Shape of merged data: (6029, 12)
Number of features for TF-IDF: 6144


## Iterative Stratification K-fold CV

To make the most of our limited multi-label data, we use Iterative Stratification K-fold CV across all our models.

In [7]:

from skmultilearn.model_selection import IterativeStratification

Y = merged_data[target_categories].fillna(False).astype(int).values

# Set up Iterative Stratification
n_splits = 5
stratifier = IterativeStratification(n_splits=n_splits, order=1)

<positron-console-cell-7>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


## Logistic Model + Random Forest using TF-IDF

In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, multilabel_confusion_matrix, precision_score, recall_score

best_params_rf = {'estimator__max_depth': None, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 75}

rf_params = {k.replace('estimator__', ''): v for k, v in best_params_rf.items()}

models = {
    "Logistic Regression": OneVsRestClassifier(LogisticRegression(max_iter=1000, C= 10.0, penalty='l1', solver='liblinear')),
    "Balanced Logistic Regression": OneVsRestClassifier(LogisticRegression(class_weight='balanced', max_iter=1000, C= 1.0, penalty='l2', solver='liblinear')),
    "Random Forest": RandomForestClassifier(**rf_params)
}

label_names = merged_data[target_categories].columns.tolist()

results = []

for model_name, model in models.items():
    print(f"\n--- {model_name} ---")

    # Store metrics per fold, per label
    fold_metrics = []

    f1_scores = []

    for train_idx, test_idx in stratifier.split(X, Y):
        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]

        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)

        Y_pred = Y_pred.toarray() if hasattr(Y_pred, 'toarray') else Y_pred

        f1_micro = f1_score(Y_test, Y_pred, average='micro')  # or 'macro'
        f1_macro = f1_score(Y_test, Y_pred, average='macro')
        # save to display later
        f1_scores.append({'f1_micro': f1_micro, 'f1_macro': f1_macro})
        f1_scores_df = pd.DataFrame(f1_scores)

        # Compute confusion matrices per label
        cm = multilabel_confusion_matrix(Y_test, Y_pred)
        precision_per_label = precision_score(Y_test, Y_pred, average=None, zero_division=0)
        recall_per_label = recall_score(Y_test, Y_pred, average=None, zero_division=0)

        # Show false posotives and false negatives per label
        for i, label in enumerate(label_names):
            tn, fp, fn, tp = cm[i].ravel()

            # Use sklearn's calculated metrics
            precision = precision_per_label[i]
            recall = recall_per_label[i]

            fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
            fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
            tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
            tnr = tn / (tn + fp) if (tn + fp) > 0 else 0

            fold_metrics.append({
                'Model': model_name,
                'Fold': len(fold_metrics) // len(label_names) + 1,
                'Label': label,
                'F1_micro': f1_micro,
                'F1_macro': f1_macro,
                'Precision': precision,
                'Recall': recall,
                'FPR': fpr,
                'FNR': fnr,
                'TPR': tpr,
                'TNR': tnr,
                'TP': tp,
                'FP': fp,
                'FN': fn,
                'TN': tn
            })

        # Convert per-fold metrics to DataFrame
        fold_metrics_df = pd.DataFrame(fold_metrics)

        # Append summary to global results list
        results.append(fold_metrics_df)

        print(f"Average F1 (micro): {np.mean(f1_scores_df.f1_micro):.4f}")
        print(f"Average F1 (macro): {np.mean(f1_scores_df.f1_macro):.4f}")
        print(classification_report(Y_test, Y_pred, target_names=label_names))


    # Combine all models into one DataFrame
    final_results_df = pd.concat(results, ignore_index=True)

    # Save to CSV
    final_results_df.to_csv('../out/model_performance_summary_tf_idf.csv', index=False)



--- Logistic Regression ---
Average F1 (micro): 0.5145
Average F1 (macro): 0.4762
                                 precision    recall  f1-score   support

                   PoliceReform       0.62      0.57      0.59       105
Operations_PatrolsInterventions       0.67      0.47      0.55        30
            StateAdministration       0.62      0.34      0.44        44
              RefugeeAssistance       0.60      0.30      0.40        20
             ElectionAssistance       0.83      0.43      0.57        23
                    LegalReform       0.47      0.27      0.34        26
         CivilSocietyAssistance       0.55      0.36      0.44        33

                      micro avg       0.62      0.44      0.51       281
                      macro avg       0.62      0.39      0.48       281
                   weighted avg       0.62      0.44      0.51       281
                    samples avg       0.09      0.08      0.08       281



C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

Average F1 (micro): 0.5178
Average F1 (macro): 0.4805
                                 precision    recall  f1-score   support

                   PoliceReform       0.65      0.51      0.57       105
Operations_PatrolsInterventions       0.67      0.52      0.58        31
            StateAdministration       0.68      0.43      0.53        44
              RefugeeAssistance       0.71      0.24      0.36        21
             ElectionAssistance       0.64      0.41      0.50        22
                    LegalReform       0.70      0.28      0.40        25
         CivilSocietyAssistance       0.47      0.44      0.45        32

                      micro avg       0.63      0.44      0.52       280
                      macro avg       0.65      0.40      0.48       280
                   weighted avg       0.64      0.44      0.52       280
                    samples avg       0.08      0.08      0.08       280



C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

Average F1 (micro): 0.5104
Average F1 (macro): 0.4689
                                 precision    recall  f1-score   support

                   PoliceReform       0.70      0.50      0.59       105
Operations_PatrolsInterventions       0.82      0.60      0.69        30
            StateAdministration       0.58      0.35      0.43        43
              RefugeeAssistance       0.60      0.30      0.40        20
             ElectionAssistance       0.60      0.26      0.36        23
                    LegalReform       0.41      0.27      0.33        26
         CivilSocietyAssistance       0.58      0.22      0.32        32

                      micro avg       0.65      0.40      0.50       279
                      macro avg       0.61      0.36      0.45       279
                   weighted avg       0.64      0.40      0.49       279
                    samples avg       0.08      0.07      0.08       279



...

C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Use

Average F1 (micro): 0.1781
Average F1 (macro): 0.1361
                                 precision    recall  f1-score   support

                   PoliceReform       0.86      0.11      0.20       105
Operations_PatrolsInterventions       1.00      0.10      0.18        31
            StateAdministration       1.00      0.23      0.37        44
              RefugeeAssistance       1.00      0.05      0.09        21
             ElectionAssistance       1.00      0.04      0.08        23
                    LegalReform       0.50      0.04      0.07        26
         CivilSocietyAssistance       0.00      0.00      0.00        32

                      micro avg       0.90      0.10      0.18       282
                      macro avg       0.77      0.08      0.14       282
                   weighted avg       0.79      0.10      0.17       282
                    samples avg       0.02      0.02      0.02       282



C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Use

Average F1 (micro): 0.1739
Average F1 (macro): 0.1313
                                 precision    recall  f1-score   support

                   PoliceReform       0.81      0.12      0.21       105
Operations_PatrolsInterventions       1.00      0.13      0.24        30
            StateAdministration       1.00      0.14      0.24        44
              RefugeeAssistance       1.00      0.05      0.10        20
             ElectionAssistance       0.00      0.00      0.00        22
                    LegalReform       0.00      0.00      0.00        25
         CivilSocietyAssistance       0.00      0.00      0.00        32

                      micro avg       0.89      0.09      0.16       278
                      macro avg       0.54      0.06      0.11       278
                   weighted avg       0.65      0.09      0.15       278
                    samples avg       0.02      0.02      0.02       278



C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\felix\miniconda3\envs\pact-ml\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Use

#### Grid Search for best parameters for logistic regression

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

# Logistic Regression Hyperparameter Grid
param_grid_lr = {
    'estimator__C': np.logspace(-3, 3, 20),
    'estimator__penalty': ['l1', 'l2'],
    'estimator__solver': ['liblinear']  # Only solver supporting l1
}

# Model wrapper
base_lr = OneVsRestClassifier(LogisticRegression(max_iter=1000))
balanced_lr = OneVsRestClassifier(LogisticRegression(class_weight='balanced', max_iter=1000))

# For storing metrics
# all_f1_scores = []

# # Grid Search for base LR
# for fold_idx, (train_idx, test_idx) in enumerate(stratifier.split(X, Y)):
#     print(f"\n📂 Fold {fold_idx + 1}/{n_splits}")

#     X_train, X_test = X[train_idx], X[test_idx]
#     y_train, y_test = Y[train_idx], Y[test_idx]

#     # Grid search
#     grid_search = GridSearchCV(base_lr, param_grid_lr, cv=3, scoring='f1_micro', verbose=1, n_jobs=-1)
#     grid_search.fit(X_train, y_train)

#     best_model = grid_search.best_estimator_
#     print(f"Best params for Base LR: {grid_search.best_params_}")

#     y_pred = best_model.predict(X_test)
#     y_pred = y_pred.toarray() if hasattr(y_pred, "toarray") else y_pred

#     f1 = f1_score(y_test, y_pred, average="micro")
#     all_f1_scores.append(f1)

#     print(f"F1 Micro: {f1:.4f}")
#     print(classification_report(y_test, y_pred, target_names=label_names))

# print(f"\n✅ Average F1 Micro over all folds: {np.mean(all_f1_scores):.4f}")

# # Grid Search for base LR
# for fold_idx, (train_idx, test_idx) in enumerate(stratifier.split(X, Y)):
#     print(f"\n📂 Fold {fold_idx + 1}/{n_splits}")

#     X_train, X_test = X[train_idx], X[test_idx]
#     y_train, y_test = Y[train_idx], Y[test_idx]

#     # Grid search
#     grid_search = GridSearchCV(balanced_lr, param_grid_lr, cv=3, scoring='f1_micro', verbose=1, n_jobs=-1)
#     grid_search.fit(X_train, y_train)

#     best_model = grid_search.best_estimator_
#     print(f"Best params for Balanced LR: {grid_search.best_params_}")

#     y_pred = best_model.predict(X_test)
#     y_pred = y_pred.toarray() if hasattr(y_pred, "toarray") else y_pred

#     f1 = f1_score(y_test, y_pred, average="micro")
#     all_f1_scores.append(f1)

#     print(f"F1 Micro: {f1:.4f}")
#     print(classification_report(y_test, y_pred, target_names=label_names))

# print(f"\n✅ Average F1 Micro over all folds: {np.mean(all_f1_scores):.4f}")

grid_search_base_lr = GridSearchCV(
    OneVsRestClassifier(LogisticRegression(max_iter=1000)),
    param_grid_lr,
    cv=stratifier,
    scoring='f1_micro',
    n_jobs=-1,
    verbose=1
)

grid_search_base_lr.fit(X, Y)
print("✅ Best parameters (global) for base LR:", grid_search_base_lr.best_params_)

grid_search_balanced_lr = GridSearchCV(
    OneVsRestClassifier(LogisticRegression(class_weight='balanced', max_iter=1000)),
    param_grid_lr,
    cv=stratifier,
    scoring='f1_micro',
    n_jobs=-1,
    verbose=1
)

grid_search_balanced_lr.fit(X, Y)
print("✅ Best parameters (global) for balanced LR:", grid_search_balanced_lr.best_params_)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
✅ Best parameters (global) for base LR: {'estimator__C': np.float64(483.2930238571752), 'estimator__penalty': 'l2', 'estimator__solver': 'liblinear'}
Fitting 5 folds for each of 40 candidates, totalling 200 fits
✅ Best parameters (global) for balanced LR: {'estimator__C': np.float64(12.742749857031322), 'estimator__penalty': 'l2', 'estimator__solver': 'liblinear'}


#### Grid Search for Random Forest Hyperparameters

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

# Random Forest Hyperparameter Grid
param_grid_rf = {
    'estimator__n_estimators': [50, 75, 100, 125, 150, 175, 200],
    'estimator__max_depth': [None, 10, 20, 30, 40, 50],
    'estimator__min_samples_split': [2, 5, 10]
}

# Model wrapper
base_rf = OneVsRestClassifier(RandomForestClassifier())

# # For storing metrics
# all_f1_scores = []

# # Grid Search for base RF
# for fold_idx, (train_idx, test_idx) in enumerate(stratifier.split(X, Y)):
#     print(f"\n📂 Fold {fold_idx + 1}/{n_splits}")

#     X_train, X_test = X[train_idx], X[test_idx]
#     y_train, y_test = Y[train_idx], Y[test_idx]

#     # Grid search
#     grid_search = GridSearchCV(base_rf, param_grid_rf, cv=3, scoring='f1_micro', verbose=1, n_jobs=-1)
#     grid_search.fit(X_train, y_train)

#     best_model = grid_search.best_estimator_
#     print(f"Best params for Random Forest: {grid_search.best_params_}")

#     y_pred = best_model.predict(X_test)
#     y_pred = y_pred.toarray() if hasattr(y_pred, "toarray") else y_pred

#     f1 = f1_score(y_test, y_pred, average="micro")
#     all_f1_scores.append(f1)

#     print(f"F1 Micro: {f1:.4f}")
#     print(classification_report(y_test, y_pred, target_names=label_names))

# print(f"\n✅ Average F1 Micro over all folds: {np.mean(all_f1_scores):.4f}")

grid_search_rf = GridSearchCV(
    OneVsRestClassifier(RandomForestClassifier()),
    param_grid_rf,
    cv=stratifier,
    scoring='f1_micro',
    n_jobs=-1,
    verbose=1
)

grid_search_rf.fit(X, Y)
print("✅ Best parameters (global) for Random Forest:", grid_search_rf.best_params_)


Fitting 10 folds for each of 126 candidates, totalling 1260 fits
✅ Best parameters (global) for Random Forest: {'estimator__max_depth': 40, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 75}
